Modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/code_agents.ipynb

Selecting a Playlist for the Party Using smolagents

In [1]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_music_recommendations = web_search(query="best party music recommendations")                               
  wayne_mansion_music = web_search(query="proposed music for Wayne's mansion party")                               
  print("Party music recommendations:", party_music_recommendations)                                               
  print("Music for Wayne's mansion party:", wayne_mansion_music)                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Party music recommendations: ## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

[50+ Best Songs For Your Next Party in 2025 - Aleka's 
Get-Together](https://alekasgettogether.com/top-songs-for-any-party/)
Whether you're planning a How To Host The Best Party At Home or just want to liven up your playlist, we've got you 
covered with 100 crowd-pleasing tunes. From timeless classics to modern hits, these songs are guaranteed to keep 
the energy high and the good times rolling all night long. 50 Popular Party Songs. Uptown Funk - Mark Ronson ft ...

[Best Songs To Party: DJ's Ultimate Party Songs Playlist - 
Top40Weekly.com](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist. If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference. From classic hits 
to ...

[The Top 100 Best Party Songs of All Time - 
LiveAbout](https://www.liveabout.com/top-best-party-songs-of-all-time-3248355)
"Macarena" then spent 14 weeks at No. 1 on the U.S. pop singles chart. For more than a year this was one of the 
most popular special event songs in the United States. It still works well as a charming party song encouraging 
everyone to join in on the simple dance.

[40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
The 40 best party songs ever made. It's party time guys - don your dancing shoes and have your speaker at the 
ready. Thursday 13 February 2025. Share. Copy Link. Facebook Twitter Pinterest ...

[The 101 Best Party Songs Of All Time - Music Grotto](https://www.musicgrotto.com/best-party-songs/)
A pounding beat and dramatic vocals make this song one of the best party songs to come out of Europe, and the 
song's music video features a strange and compelling premise of a dead man being reanimated and falling in love 
with a woman. Tjis Verwest, (a.k.a. Tiësto) composed the song with help from James Bell, Julia Karlsson, and Anton 
Rundberg.

[Best Party Playlist: 25 High-Energy Songs for the Ultimate 
Celebration](https://audioassemble.com/best-party-playlist/)
Whether you're planning a weekend house party, a birthday bash, or an impromptu dance-off, every celebration 
deserves the perfect soundtrack. Our selection of 25 high-energy songs are tried-and-true crowd pleasers that 
seamlessly blend different genres and eras, ensuring there's something for everyone.

[Best Party Songs — Best Dance Music - 
Cosmopolitan](https://www.cosmopolitan.com/uk/entertainment/a46785812/best-songs-for-party-playlist/)
3. 'Taki Taki,' by DJ Snake, feat. Selena Gomez, Ozuna, and Cardi B. If both Selena Gomez and Cardi B are featured 
on a track, it just has to be a good one.This powerful Latinx collab was made ...

[The Best Party Songs of All Time (Our Playlists)](https://www.ispytunes.com/post/best-party-songs)
Choosing the best party songs can turn your event into an unforgettable experience. Whether you're looking for 
timeless classics or the latest hits, a well-crafted playlist sets the mood and keeps the energy high. From disco 
beats to pop anthems and hip-hop jams, these tracks bring people together on the dance floor. ...

[100 Best Party Songs: The Ultimate Party Playlist](https://audiocaptain.com/best-party-songs/)
Best 2000s Party Songs. Stylistically speaking, music that dominated the charts in the 2000s was nondescript.. 
There was not much new regarding musical directions. C

[Step 0: Duration 9.01 seconds| Input tokens: 2,085 | Output tokens: 124]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(recommended_songs)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['September - Earth, Wind & Fire', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'Macarena - Los 
Del Rio', 'Taki Taki - DJ Snake, Selena Gomez, Ozuna, Cardi B', 'Mansion Party - Tee Grizzley, Fredo Bang', "VIP - 
Amanda La'Depeche, David Guetta", 'Roar - Katy Perry', 'Party in the USA - Miley Cyrus', 'Shake It Off - Taylor 
Swift', "Can't Stop the Feeling! - Justin Timberlake"]

[Step 2: Duration 5.95 seconds| Input tokens: 11,512 | Output tokens: 483]

['September - Earth, Wind & Fire',
 'Uptown Funk - Mark Ronson ft. Bruno Mars',
 'Macarena - Los Del Rio',
 'Taki Taki - DJ Snake, Selena Gomez, Ozuna, Cardi B',
 'Mansion Party - Tee Grizzley, Fredo Bang',
 "VIP - Amanda La'Depeche, David Guetta",
 'Roar - Katy Perry',
 'Party in the USA - Miley Cyrus',
 'Shake It Off - Taylor Swift',
 "Can't Stop the Feeling! - Justin Timberlake"]

Using Python Imports Inside the Agent


In [1]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Define the task durations in minutes                                                                           
  task_durations = [30, 60, 45, 45]                                                                                
                                                                                                                   
  # Calculate the total time required in minutes                                                                   
  total_time_minutes = sum(task_durations)                                                                         
                                                                                                                   
  # Convert total time to a timedelta                                                                              
  total_time = datetime.timedelta(minutes=total_time_minutes)                                                      
                                                                                                                   
  # Get the current time                                                                                           
  current_time = datetime.datetime.now()                                                                           
                                                                                                                   
  # Calculate the end time                                                                                         
  end_time = current_time + total_time                                                                             
                                                                                                                   
  # Print the end time in a readable format                                                                        
  print("Alfred will be ready by:", end_time.strftime("%Y-%m-%d %H:%M:%S"))                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Alfred will be ready by: 2025-02-28 00:23:08

Out: None

[Step 0: Duration 10.56 seconds| Input tokens: 2,096 | Output tokens: 198]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Alfred will be ready by: 2025-02-28 00:23:08")                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Alfred will be ready by: 2025-02-28 00:23:08

[Step 1: Duration 4.56 seconds| Input tokens: 4,628 | Output tokens: 274]

'Alfred will be ready by: 2025-02-28 00:23:08'

Sharing Our Custom Party Preparator Agent to the Hub


In [1]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To create the best playlist for a villain masquerade party at Wayne's mansion, I should first get ideas   
for the music that fits the theme. Since this is a villain masquerade, I will search for popular villain theme     
songs and music compilations that suit the occasion.                                                               
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
villain_theme_music = web_search(query="villain masquerade theme music")                                           
print(villain_theme_music)                                                                                         
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  villain_theme_music = web_search(query="villain masquerade theme music")                                         
  print(villain_theme_music)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Devils Masquerade - Villain Anthem | Dark Cinematic Music](https://www.youtube.com/watch?v=aAXswzON6q8)
🔥 **"Devils Masquerade " - The Ultimate Villain Anthem!** 🔥 This dark villain anthem sets the stage for power, 
chaos, and domination. Sinister melodies, h...

[Villain Music | No Copyright Song & MP3 Free Downloads - Pixabay](https://pixabay.com/music/search/villain/)
82+ Villain no copyright music Download villain royalty-free audio tracks and instrumentals for your next project. 
dark. cinematic. epic. hero. ai generated. evil. fantasy. music. halloween. Royalty-free music tracks. A Villain's 
Redemption Katzen_Tupas. 1:55. Solo guitar. Villain Hero. 107. Epic Sci-Fi Villain Theme for Cinematic Projects ...

[10 Scariest Antagonist Theme Songs in Movies, According to 
Reddit](https://collider.com/scariest-antagonist-theme-songs-reddit/)
Pemberton's instrumental theme for villain The Prowler is particularly memorable, having first appeared in Into the
Spider-Verse. Related The 10 Worst Netflix Shows, According to Reddit

[Favourite Villain Themes : r/soundtracks - 
Reddit](https://www.reddit.com/r/soundtracks/comments/mbp992/favourite_villain_themes/)
The best theme ever made for a Villain is the "Occupation Of Balmora - Epic Edition" aka for non SWEU fans Darth 
Revans Theme. Its grandiose, terrifying and makes you feel like the true villain of star wars. Imperial March is to
this what a fly is to a boot. Nothing. And bits of it are used as the Sith Warriors theme in swtor.

[Villain Theme Music | No Copyright Song & MP3 Free Downloads - 
Pixabay](https://pixabay.com/music/search/villain+theme/)
1,985+ Villain Theme no copyright music Download villain theme royalty-free audio tracks and instrumentals for your
next project. theme. intro. background. music. cinematic. villain. ai generated. epic. orchestral. dark. 
Royalty-free music tracks. Epic Orchestral Villain Theme nickpanek620. 1:27. Main title.

[Looking for recommendations for a villain's theme 
song](https://www.reddit.com/r/DungeonsAndDragons/comments/10z77cg/looking_for_recommendations_for_a_villains_theme
/)
I'm searching for a theme song to play during "cutscenes" or certain monologues, preferibly something that is 
classic, isn't too famous, and is played on a violin. So far, Hunter X Hunter's Shaiapouf theme seems very close to
the kind of theme I want (it's tragic, ominous and very aristocratic), but I'm looking for more variety in options.

[10 Best Marvel Movie Villain Theme Songs That Live Rent Free In Our 
Heads](https://screenrant.com/best-marvel-villain-theme-songs/)
Movies of the MCU and other cinematic Marvel properties have had some amazing original soundtracks, particularly 
when it comes to the villains' themes. While licensed music from famous artists usually gets most of the spotlight 
when it comes to superhero movies, the Marvel Cinematic Universe has some amazing original songs worthy of special 
praise. . Specifically, the various villains of ...

[The 99 Best Songs for Villains (Our Villain Arc Playlist) - 
iSpyTunes](https://www.ispytunes.com/post/best-songs-for-villains)
Enter the captivating world of villain songs with our curated playlist of the 99 best songs for villains, where 
darkness and power converge. top of page. Home. Submit a Song. SOTW Winners; ... Theme Song) The Wrath of Khan 
(Star Trek - Theme Song) The World's Greatest Criminal Mind by Vincent Price (The Great Mouse Detective)

[Top Ten Villain Musical Themes - CINERAMBLE](https://cineramble.com/2015/10/top-ten-villain-musical-themes/)
THE KRAKEN THEME from PIRATES OF THE CARIBBEAN: DEAD MAN'S CHEST (2006) Composed by Hans Zimmer. Han Zimmer's 
scores for the Pirates of the Caribbean series are just as wild as the films themselves. Mixing a lot of different 
types of styles together, from classical to heavy metal, his music is a perfect accompaniment for this 
unconventional series.

[Top 25 Boss and Villain Soundtracks and Theme 
Son

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The playlist looks great and represents a variety of themes and genres fitting for Wayne's mansion        
masquerade party. Before finalizing, I will make sure the format is consistent and the theme titles are properly   
structured to ensure clarity and ease of use.                                                                      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
playlist = [                                                                                                       
    "Devils Masquerade - Villain Anthem",                                                                          
    "The Ultimate Villain Anthem by Katzen_Tupas",                                                                 
    "Epic Sci-Fi Villain Theme by nickpanek620",                                                                   
    "Imperial March (Darth Vader) by John Williams",                                                               
    "The Wrath of Khan (Star Trek) Theme Song",                                                                    
    "Dancing Mad (Full Song) from Final Fantasy VI",                                                               
    "Kraken Theme from Pirates of the Caribbean: Dead Man's Chest by Hans Zimmer",                                 
    "Occupation Of Balmora - Epic Edition (Darth Revan's Theme from SWTOR)",                                       
    "Hunter X Hunter's Shaiapouf theme by Kim TaeHyung (BTSmember)"                                                
]                                                                                                                  
                                                                                                                   
playlist_formatted = "\n".join(f"- {song}" for song in playlist)                                                   
final_answer(playlist_formatted)                                                                                   
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist = [                                                                                                     
      "Devils Masquerade - Villain Anthem",                                                                        
      "The Ultimate Villain Anthem by Katzen_Tupas",                                                               
      "Epic Sci-Fi Villain Theme by nickpanek620",                                                                 
      "Imperial March (Darth Vader) by John Williams",                                                             
      "The Wrath of Khan (Star Trek) Theme Song",                                                                  
      "Dancing Mad (Full Song) from Final Fantasy VI",                                                             
      "Kraken Theme from Pirates of the Caribbean: Dead Man's Chest by Hans Zimmer",                               
      "Occupation Of Balmora - Epic Edition (Darth Revan's Theme from SWTOR)",                                     
      "Hunter X Hunter's Shaiapouf theme by Kim TaeHyung (BTSmember)"                                              
  ]                                                                                                                
                                                                                                                   
  playlist_formatted = "\n".join(f"- {song}" for song in playlist)                                                 
  final_answer(playlist_formatted)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: - Devils Masquerade - Villain Anthem
- The Ultimate Villain Anthem by Katzen_Tupas
- Epic Sci-Fi Villain Theme by nickpanek620
- Imperial March (Darth Vader) by John Williams
- The Wrath of Khan (Star Trek) Theme Song
- Dancing Mad (Full Song) from Final Fantasy VI
- Kraken Theme from Pirates of the Caribbean: Dead Man's Chest by Hans Zimmer
- Occupation Of Balmora - Epic Edition (Darth Revan's Theme from SWTOR)
- Hunter X Hunter's Shaiapouf theme by Kim TaeHyung (BTSmember)

[Step 2: Duration 12.81 seconds| Input tokens: 10,266 | Output tokens: 580]

"- Devils Masquerade - Villain Anthem\n- The Ultimate Villain Anthem by Katzen_Tupas\n- Epic Sci-Fi Villain Theme by nickpanek620\n- Imperial March (Darth Vader) by John Williams\n- The Wrath of Khan (Star Trek) Theme Song\n- Dancing Mad (Full Song) from Final Fantasy VI\n- Kraken Theme from Pirates of the Caribbean: Dead Man's Chest by Hans Zimmer\n- Occupation Of Balmora - Epic Edition (Darth Revan's Theme from SWTOR)\n- Hunter X Hunter's Shaiapouf theme by Kim TaeHyung (BTSmember)"

In [3]:
agent.push_to_hub('CoralLeiCN/agentcourse_AlfredAgent')

README.md:   0%|          | 0.00/243 [00:00<?, ?B/s]

This agent has step_callbacks: they will be ignored by this method. 1

CommitInfo(commit_url='https://huggingface.co/spaces/CoralLeiCN/agentcourse_AlfredAgent/commit/ceae35f20b2a40c0e65961f3745925e489e32021', commit_message='Upload agent', commit_description='', oid='ceae35f20b2a40c0e65961f3745925e489e32021', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/CoralLeiCN/agentcourse_AlfredAgent', endpoint='https://huggingface.co', repo_type='space', repo_id='CoralLeiCN/agentcourse_AlfredAgent'), pr_revision=None, pr_num=None)